## Update a wflow model: landuse

Once you have a **Wflow** model, you may want to update your model in order to use a new landuse map, change a parameter value, add sample locations, use different forcing data, create and run different scenarios etc.

With HydroMT, you can easily read your model and update one or several components of your model using the **update** function of the command line interface (CLI). Here are the steps and some examples on how to **update the landuse map and parameters**.

All lines in this notebook which starts with ! are executed from the command line. Within the notebook environment the logging messages are shown after completion. You can also copy these lines and paste them in your shell to get more feedback.

### Import packages

In this notebook, we will use some functions of HydroMT to check available datasets and also to plot the landuse maps from the original and updated models. Here are the libraries to import to realize these steps.

In [ ]:
# for plotting
import matplotlib.pyplot as plt
from matplotlib import cm
import cartopy.crs as ccrs
proj = ccrs.PlateCarree() # plot projection

In [ ]:
# import hydromt
import hydromt

In [ ]:
# setup logging
from  hydromt.log import setuplog
logger = setuplog("update_model_landuse", log_level=10)

### Searching the data catalog for landuse

In our previous notebook, we built a wflow model using the *GlobCover* landuse classification. But we could as well have chosen another one. Let's what other landuse landcover data are available in HydroMT and choose another landuse classification for our model. For this we will open the data catalog.

You can also directly open and search the HydroMT yaml library by downloading and opening the data_catalog.yml file in [hydromt-artifacts](https://github.com/DirkEilander/hydromt-artifacts/releases/tag/v0.0.3) or look at the list of data sources in the [documentation](https://deltares.github.io/hydromt/latest/user_guide/data.html#available-global-datasets).

In [ ]:
# Load the default data catalog of HydroMT
data_catalog = hydromt.DataCatalog(logger=logger)

In [ ]:
# Check which landuse/lancover sources are available in the DataCatalog
data_table = data_catalog.to_dataframe()
data_table.source_url[data_table['category'] == 'landuse & landcover']

Here we can see that we have five data sources in HydroMT related to landuse & landcover properties. Out of these, three are landuse classifications:

- **globcover** (already used in our current model)
- **corine**
- **vito**

The other two are a Leaf Area Index data source (modis_lai) and a canopy height map (simard).

Let's now see how to update our current model in one command line to use the **corine** classification.

### hydroMT CLI update interface

Using the `hydroMT build` API, we can update one or several components of an already existing Wflow model. Let's get an overview of the available options:

In [ ]:
# Print the options available from the update command
! hydromt update --help

### Update wflow landuse layers

In [ ]:
# NOTE: copy this line (without !) to your shell for more direct feedback
! hydromt update wflow wflow_piave_subbasin -o ./wflow_piave_corine -c setup_lulcmaps --opt lulc_fn=corine -v

The example above means the following: run **hydromt** with:

- `update wflow`: i.e. update a wflow model
- `wflow_piave_subbasin`: original model folder
- `-o ./wflow_piave_corine`: output updated model folder
- `-c setup_lulcmaps`: model component to update, here setup_lulcmaps for landuse layers
- `--opt lulc_fn=corine`: arguments to use when updating the setup_lulcmaps component, all options are described in the [docs(model_components)](https://deltares.github.io/hydromt_wflow/latest/user_guide/wflow/components.html)
- `v`: give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.

### Model comparison

From the information above, you can see that not only was the landuse map updated but also all wflow landuse-related parameters with it: Kext, N, PathFrac, RootingDepth, Sl, Swood and WaterFrac.

Let's now have a look at the different landuse maps of our two models and check that they were indeed updated.

In [ ]:
# Load both models with hydromt
mod0 = hydromt.WflowModel(root='wflow_piave_subbasin', mode='r', logger=logger)
mod1 = hydromt.WflowModel(root='wflow_piave_corine', mode='r', logger=logger)

In [ ]:
# plot  globcover map
# create new figure
fig = plt.figure(figsize=(10,5.5))
ax = fig.add_subplot(projection=proj)
# plot globcover landuse
mask = mod0.staticmaps['wflow_subcatch']>0
mod0.staticmaps['wflow_landuse'].where(mask).plot(
    ax=ax, cmap=cm.Set1
)

In [ ]:
# plot  corine map
# create new figure
fig = plt.figure(figsize=(10,5.5))
ax = fig.add_subplot(projection=proj)
# plot globcover landuse
mask = mod1.staticmaps['wflow_subcatch']>0
mod1.staticmaps['wflow_landuse'].where(mask).plot(
    ax=ax, cmap=cm.Set1
)